In [ ]:
import os
from glob import glob
import random
import copy
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import ndimage
import cv2
from PIL import Image
from tqdm.notebook import tqdm
import radiomics
from radiomics import featureextractor
import SimpleITK as sitk
import six

In [ ]:
IMG_SIZE = (150, 150) # width, height
MASK_THRESH = 100

def add_padding(img):
    height, width, _ = img.shape
    img_width, img_height = IMG_SIZE
    fill_value = (0, 0, 0)
    if width / height == img_width / img_height:
        img = cv2.resize(img, (IMG_SIZE))
    elif width / height < img_width / img_height:
        tmp_width = math.ceil(width / (height / img_height))
        img = cv2.resize(img, (tmp_width, img_height))
        to_add = img_width - tmp_width
        left = to_add // 2
        right = to_add - left
        img = cv2.copyMakeBorder(img, 0, 0, left, right, cv2.BORDER_CONSTANT, value=fill_value)
    else:
        tmp_height = math.ceil(height / (width / img_width))
        img = cv2.resize(img, (img_width, tmp_height))
        to_add = img_height - tmp_height
        top = to_add // 2
        bottom = to_add - top
        img = cv2.copyMakeBorder(img, top, bottom, 0, 0, cv2.BORDER_CONSTANT, value=fill_value)
    return img

In [ ]:
remove_col_names = [
    'diagnostics_Versions_PyRadiomics',
    'diagnostics_Versions_Numpy',
    'diagnostics_Versions_SimpleITK',
    'diagnostics_Versions_PyWavelet',
    'diagnostics_Versions_Python',
    'diagnostics_Configuration_Settings',
    'diagnostics_Configuration_EnabledImageTypes',
    'diagnostics_Image-original_Hash',
    'diagnostics_Image-original_Dimensionality',
    'diagnostics_Image-original_Spacing',
    'diagnostics_Image-original_Size',
    'diagnostics_Image-original_Mean',
    'diagnostics_Image-original_Minimum',
    'diagnostics_Image-original_Maximum',
    'diagnostics_Mask-original_Hash',
    'diagnostics_Mask-original_Spacing',
    'diagnostics_Mask-original_Size',
    'diagnostics_Mask-original_BoundingBox',
    'diagnostics_Mask-original_VoxelNum',
    'diagnostics_Mask-original_VolumeNum',
    'diagnostics_Mask-original_CenterOfMassIndex',
    'diagnostics_Mask-original_CenterOfMass',
]

In [ ]:
data_path = 'data_path.csv'

df = pd.read_csv(f'/workspace/data4/changwoo/Colonoscopy/XAI_test/XAI_test.csv')
img_list = df['img_path'].tolist()

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor()
radiomics_df = pd.DataFrame()

for idx in tqdm(range(len(img_list))):
    img_path = img_list[idx]
    img_name = img_path.split('/')[-1].split('.')[0]
    img = cv2.imread(img_path)
    img_h, img_w, _ = img.shape
    xmin, xmax, ymin, ymax = df.loc[df['img_path']==img_path, ['xmin', 'xmax', 'ymin', 'ymax']].values[0]
    img = img[ymin:ymax, xmin:xmax, :]
    img = add_padding(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = sitk.GetImageFromArray(img)
    
    mask_path = f'cam_mask_{idx}_png'
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    mask[np.where(mask <= MASK_THRESH)] = 0
    mask[np.where(mask > MASK_THRESH)] = 1
    mask = sitk.GetImageFromArray(mask)
    
    try:
        result = extractor.execute(img, mask)
    except:
        continue

    result_df = pd.DataFrame.from_dict(result, orient='index').T
    result_df['img_name'] = img_path.split('/')[-1]

    if radiomics_df.shape[0] == 0:
        radiomics_df = result_df
    else:
        radiomics_df = pd.concat([radiomics_df, result_df])

radiomics_df = radiomics_df[['img_name'] + [col for col in radiomics_df.columns[:-1] if col not in remove_col_names]].reset_index(drop=True)